<a href="https://colab.research.google.com/github/kevinmcaleer/inverse_kinematics/blob/main/Robot_Arm_with_Sliders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the libraries

Import NumPy for helping with some Math, and MatPlotLib for plotting the Arm on a chart.

In [10]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt

# Make the Notebook interactive with widgets
from ipywidgets import interact, FloatSlider, FloatText
import ipywidgets as widgets

# Calculate the Inverse Kinematics

Calculate the IK for a given end point (x,y), for each arm section.

In [5]:
# Inverse Kinematics calculation
def inverse_kinematics(x, y, l1, l2, l3):
    """ Calculate the IK for arm sections l1, l2, and l3 using the x & y target """
    r = np.sqrt(x**2 + y**2)
    r_adj = r - l3
    cos_angle2 = (r_adj**2 - l1**2 - l2**2) / (2 * l1 * l2)
    angle2 = np.arccos(np.clip(cos_angle2, -1, 1))
    k1 = l1 + l2 * np.cos(angle2)
    k2 = l2 * np.sin(angle2)
    angle1 = np.arctan2(y, x) - np.arctan2(k2, k1)
    angle3 = np.arctan2(y, x) - angle1 - angle2

    # return the angles for each joint
    return np.degrees(angle1), np.degrees(angle2), np.degrees(angle3)

# Plot the Chart

This function plots the arm on a chart.

In [6]:
def plot_arm(x, y, l1, l2, l3):
    """ Plot the Arm on the chart """
    angles = inverse_kinematics(x, y, l1, l2, l3)
    angle1, angle2, angle3 = np.radians(angles)

    joint1 = (l1 * np.cos(angle1), l1 * np.sin(angle1))
    joint2 = (joint1[0] + l2 * np.cos(angle1 + angle2), joint1[1] + l2 * np.sin(angle1 + angle2))
    effector = (joint2[0] + l3 * np.cos(angle1 + angle2 + angle3), joint2[1] + l3 * np.sin(angle1 + angle2 + angle3))

    plt.figure(figsize=(6, 6))
    plt.plot([0, joint1[0]], [0, joint1[1]], 'ro-')  # Base to joint1
    plt.plot([joint1[0], joint2[0]], [joint1[1], joint2[1]], 'go-')  # joint1 to joint2
    plt.plot([joint2[0], effector[0]], [joint2[1], effector[1]], 'bo-')  # joint2 to effector
    plt.plot(x, y, 'yx')  # Target position
    plt.scatter([effector[0]], [effector[1]], s=100, facecolors='none', edgecolors='r')  # Crosshair at end effector

    # Angle annotations
    plt.text(0.5 * joint1[0], 0.5 * joint1[1], f'{np.degrees(angle1):.2f}°', color='red')
    plt.text(joint1[0] + 0.5 * (joint2[0] - joint1[0]), joint1[1] + 0.5 * (joint2[1] - joint1[1]), f'{np.degrees(angle2):.2f}°', color='green')
    plt.text(joint2[0] + 0.5 * (effector[0] - joint2[0]), joint2[1] + 0.5 * (effector[1] - joint2[1]), f'{np.degrees(angle3):.2f}°', color='blue')

    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Robotic Arm Position')
    plt.grid(True)
    plt.show()

In [9]:
# Interactive controls
x_slider = FloatSlider(min=-2, max=2, step=0.1, value=1, description='X Position:')
y_slider = FloatSlider(min=-2, max=2, step=0.1, value=1, description='Y Position:')
l1_text = FloatText(value=1, description='L1 Length:')
l2_text = FloatText(value=1, description='L2 Length:')
l3_text = FloatText(value=1, description='L3 Length:')

# Play time
interact(plot_arm, x=x_slider, y=y_slider, l1=l1_text, l2=l2_text, l3=l3_text)